In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import neurd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
from neurd.vdi_microns import volume_data_interface as vdi
vdi.set_parameters_for_directory_modules()

In [11]:
mesh_dir = "../autoproof_neuron_pipeline/"

vdi.neuron_obj_auto_proof_directory = mesh_dir
vdi.neuron_obj_directory = mesh_dir
vdi.meshes_directory = mesh_dir

# Loading the annotated neurons

In [4]:
segment_id_post = 864691134917511946
segment_id_pre = 864691135567721964

In [12]:
neuron_obj_post = vdi.load_neuron_obj_auto_proof(
    segment_id_post
)

neuron_obj_pre = vdi.load_neuron_obj_auto_proof(
    segment_id_pre
)

Decompressing Neuron in minimal output mode...please wait
Decompressing Neuron in minimal output mode...please wait


In [14]:
from neurd import neuron_visualizations as nviz

nviz.plot_compartments(neuron_obj_pre)


 Working on visualization type: mesh


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

apical:blue
apical_shaft:aqua
apical_tuft:purple
basal:yellow
axon:red
oblique:green


Unlabeled: transparent green


In [15]:
nviz.plot_compartments(neuron_obj_post)


 Working on visualization type: mesh


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

apical:blue
apical_shaft:aqua
apical_tuft:purple
basal:yellow
axon:red
oblique:green


Unlabeled: transparent green


# Calculating the proximities

In [ ]:
presyn_prox_data = pxu.presyn_proximity_data(
    segment_id = segment_id,
    split_index = split_index,
    plot = False,
    verbose = False
)

In [ ]:
def presyn_proximity_data(
    segment_id,
    split_index,
    plot = False,
    verbose = False,
    ):
    """
    Purpose: Get the presyn proximity information
    before pairwise proximities are computed
    """
    st = time.time()
    #1) computes the synapse presyn side of the connectome
    synapse_pre_df = vdi.segment_id_to_synapse_table_optimized_connectome(
        segment_id=segment_id,
        split_index=split_index,
        syn_type = "presyn",
        coordinates_nm = True,
        return_df=True)
    
    synapse_pre_raw_df = vdi.segment_id_to_synapse_table_optimized(
        segment_id=segment_id,
        synapse_type = "presyn",
        coordinates_nm = True,
        return_df=True)
    
    synapse_pre_raw_coords = pxu.synapse_coordinates_from_df(synapse_pre_raw_df)
    
    synapse_pre_proof_df = vdi.segment_id_to_synapse_table_optimized_proofread(
        segment_id=segment_id,
        synapse_type = "presyn",
        coordinates_nm = True,
        return_df=True)
    
    synapse_pre_proof_coords = pxu.synapse_coordinates_from_df(synapse_pre_proof_df)


    #2) Gets the mesh for later plotting
    if plot:
        mesh = vdi.fetch_proofread_mesh(
            segment_id,
            split_index)
    else:
        mesh = None

    #3) Gets some coordinate for later euclidean distance calculation
    presyn_soma_coord = vdi.soma_nm_coordinate(
        segment_id,
        split_index)

    #4) Fetches neuron object  for calculations
    G_presyn = vdi.graph_obj_from_proof_stage(segment_id,split_index)


    #5) Calculating all of the width,compartments and skeletons
    (presyn_skeleton_array,
    presyn_width_array,
    presyn_comp_array) = nxu.skeleton_width_compartment_arrays_from_G(
        G_presyn,
        compartments=["axon"],
    )

    #5) Fetches the skeleton and starting coordinate of axon 
    #   to be used later for skeletal distance to soma

    presyn_starting_coordinate = nxu.most_upstream_node_on_axon_limb(
        G_presyn,
        return_endpoint_upstream = True
    )

    if plot:
        print(f"Plotting: Presyn axon skeleton")
    (pre_sk_vert,
    pre_sk_edges)= nxu.axon_skeleton(
        G = G_presyn,
        include_path_to_soma = True,
        plot=plot,
        mesh = mesh,

    )
    
    #G_presyn_sk = sk.convert_skeleton_to_graph(pre_sk_vert[pre_sk_edges])
    G_presyn_sk = xu.graph_from_non_unique_vertices_edges(pre_sk_vert,pre_sk_edges)

    if plot:
        print(f"Plotting: Presyn skeleton and starting coordinate")
        nviz.plot_objects(
            scatters=[pre_sk_vert,presyn_starting_coordinate],
            scatters_colors=["red","blue"],
            scatter_size=[0.2,2]
        )
        
    if verbose:
        print(f"Total time for presyn proximity data = {time.time() - st}")

    return dict(
        # for the synapses to be checked if in vicinity
        synapse_pre_df = synapse_pre_df,

        # for attributes for computing proximities
        presyn_skeleton_array=presyn_skeleton_array,
        presyn_width_array=presyn_width_array,
        presyn_comp_array=presyn_comp_array,


        # for skeletal walk distance
        presyn_starting_coordinate=presyn_starting_coordinate,
        pre_sk_vert=pre_sk_vert,
        pre_sk_edges=pre_sk_edges,
        G_presyn_sk=G_presyn_sk,

        # for computing euclidean distance
        presyn_soma_coord =presyn_soma_coord,
        
        # for more features to compute close to
        synapse_pre_raw_coords=synapse_pre_raw_coords,
        synapse_pre_proof_coords=synapse_pre_proof_coords,
        
        #for ploting 
        mesh = mesh
    )

In [ ]:

def postsyn_proximity_data(
    segment_id,
    split_index,
    plot = False,
    verbose = False,
    check_starting_coord_match_skeleton = False,
    
    ):

    """
    Purpose: Get the postsyn proximity information
    before pairwise proximities are computed
    """
    segment_id_target = segment_id
    split_index_target = split_index

    st = time.time()
    #2) Gets the mesh for later plotting
    if plot:
        mesh_post = vdi.fetch_proofread_mesh(
            segment_id_target,
            split_index_target
        )
    else:
        mesh_post = None

    #3) Gets some coordinate for later euclidean distance calculation
    postsyn_soma_coord = vdi.soma_nm_coordinate(
        segment_id_target,
        split_index=split_index_target)

    #4) Fetches neuron object  for calculations
    G_postsyn = vdi.graph_obj_from_proof_stage(segment_id_target,split_index_target)

    #4) Calculating all the skeleton arrays, widths, compartments
    (postsyn_skeleton_array,
    postsyn_width_array,
    postsyn_comp_array) = nxu.skeleton_width_compartment_arrays_from_G(
        G_postsyn,
        plot=False
    )

    postsyn_soma_mesh = vdi.fetch_soma_mesh(
        segment_id_target,
        split_index_target,
        plot_soma = False)

    soma_array = np.array(postsyn_soma_mesh.vertices)
    postsyn_skeleton_array = np.vstack([postsyn_skeleton_array,postsyn_soma_mesh.vertices])
    postsyn_width_array = np.hstack([postsyn_width_array,np.repeat([0],len(soma_array))])
    postsyn_comp_array = np.hstack([postsyn_comp_array,np.repeat(["soma"],len(soma_array))])

    if plot:
        print(f"Plotting: postsyn skeleton array")
        ipvu.plot_mesh_with_scatter(
            mesh_post,
            scatter=postsyn_skeleton_array,
        )

    #5) Fetches the skeleton and starting coordinate of axon 
    #   to be used later for skeletal distance to soma

    if plot:
        print(f"Plotting: postsyn skeleton")
    (post_sk_vert,
    post_sk_edges)= nxu.skeleton(
        G = G_postsyn,
        plot=plot,
        mesh = mesh_post,
        )  

    postsyn_starting_coords= nxu.starting_coordinates_all_limbs(
                G_postsyn,
                verbose = False)


    if check_starting_coord_match_skeleton:
        for k in postsyn_starting_coords.reshape(-1,3):
            min_dist = nu.closest_dist_between_coordinates(
                post_sk_vert.reshape(-1,3),
                k.reshape(-1,3),
                return_min=True
            )

            if min_dist > 0:
                raise Exception("")

    #6) Postsyn and spine data
    synapse_post_df = vdi.segment_id_to_synapse_table_optimized_proofread(
        segment_id=segment_id_target,
        split_index=split_index_target,
        syn_type = "postsyn",
        coordinates_nm = True,
        return_df=True)

    synapse_post_coord = synapse_post_df[
        ["synapse_x_nm","synapse_y_nm","synapse_z_nm"]].to_numpy().astype('float')

    if plot:
        print(f"Plotting: Spine Coordinates")
    spine_shaft_coords = nxu.spine_shaft_coordinates(
        G_postsyn,
        verbose = False,
        plot = plot,
        mesh = mesh_post,
    )

    if verbose:
        print(f"Total time for presyn proximity data = {time.time() - st}")

    return_dict = dict(
        # for attributes for computing proximities
        postsyn_skeleton_array = postsyn_skeleton_array,
        postsyn_width_array = postsyn_width_array,
        postsyn_comp_array = postsyn_comp_array,

        # for skeletal walk distance
        post_sk_vert=post_sk_vert,
        post_sk_edges=post_sk_edges,
        postsyn_starting_coords=postsyn_starting_coords,

        # for computing euclidean distance
        postsyn_soma_coord=postsyn_soma_coord,

        # attributes to be collected in the vicinity of contact
        synapse_post_coord=synapse_post_coord,
        spine_shaft_coords=spine_shaft_coords,
        
        #for ploting 
        mesh_post = mesh_post,
        G_postsyn_sk = None,

    )
    
    return return_dict


In [ ]:
prox_data = pxu.proximity_pre_post(
    segment_id_pre = segment_id,
    split_index_pre = split_index,

    segment_id_post = segment_id_post,
    split_index_post = split_index_post,

    presyn_prox_data = presyn_prox_data,
    postsyn_prox_data = None,

    plot_proximities = False,
    verbose = verbose_inner,
    return_df = False
)